
## Step 1 - Convert hex to base64

> The string:
> ```
> 49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d
> ```
> Should produce:
> ```
> SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
> ```
> So go ahead and make that happen. You'll need to use this code for the rest of the exercises.
>
> ### Comment
>
> Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.

In [34]:
# your code here ... (put some comments to explain what you did)
from base64 import b64encode, b64decode

str = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"

def hexToBase64(input: str) -> str:
    #Used Python Library to get bytes from hex, then encode them into base64 and use ASCII for prettyprint
    base64 = b64encode(bytes.fromhex(input)).decode()
    return base64

hexToBase64(str)

'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

## Step 2 - Fixed XOR

> Write a function that takes two equal-length buffers and produces their XOR combination.
>
> If your function works properly, then when you feed it the string:
> ```
> 1c0111001f010100061a024b53535009181c
> ```
> ... after hex decoding, and when XOR'd (bitwise) against:
> ```
> 686974207468652062756c6c277320657965
> ```
> ... should produce:
> ```
> 746865206b696420646f6e277420706c6179
> ```

In [35]:
# your code with comments ... (feel free to add as many as helper functions as you need!)
def hex_to_binary(hex_str: str) -> str:
    #Convert a hexadecimal string to a binary string.
    binary_str = bin(int(hex_str, 16))[2:]
    return binary_str.zfill(len(hex_str) * 4)

def binary_to_hex(binary_str: str) -> str:
    #Convert a binary string to a hexadecimal string.
    binary_str = binary_str.zfill((len(binary_str) + 3) // 16)
    hex_str = hex(int(binary_str, 2))[2:]
    return hex_str

def fixed_xor(first_buffer: str, second_buffer: str) -> str:
    #Perform a fixed XOR on two hexadecimal strings.
    binary_first = hex_to_binary(first_buffer)
    binary_second = hex_to_binary(second_buffer)
    
    # Ensure the input strings are of equal length
    if len(binary_first) != len(binary_second):
        raise ValueError("Input strings must be of equal length")

    # Perform bitwise XOR on each pair of corresponding bits
    result_binary = ''.join('1' if bit1 != bit2 else '0' for bit1, bit2 in zip(binary_first, binary_second))
    
    # Convert the binary result back to hex
    hex_result = binary_to_hex(result_binary)
    return hex_result

first_buffer = '1c0111001f010100061a024b53535009181c'
second_buffer = '686974207468652062756c6c277320657965'

result = fixed_xor(first_buffer, second_buffer)
print(result)


746865206b696420646f6e277420706c6179


## Step 3 - Single-byte XOR cipher

> The hex encoded string:
> ```
> 1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736
> ```
> ... has been XOR'd against a single character. Find the key (which is one byte) and decrypt the message. The message is a meaningful sentence in English!
>
> You should write a code to find the key and decrypt the message. Don't do it manually!
>
> ### Comment
> There are several mini steps to achieve this! First, you need a strategy for searching in the key space. Second, you need a test/scoring mechanism to check whether the decrypted message is  meaningful or not (i.e., detecting garbage vs. the correct output). You can read more about *"Caesar"* cipher to get some ideas and more background!

#### Description
*A brief description of your approach. Don't just put the code. First explain what you did and WHY you did it!*

<p> (your description)<br>
My strategy was to bruteforce through the key space (lowercase and uppercase english letters), and XOR the bitstring with each key I'm searching. I then defined a function that gave the decoded text a score depending on how many common english words it contained, and used that as my metric to see if a decypted message is meaningful. Then I simply returned the decrypted text and key that gave the highest score
</p>

In [426]:
def single_byte_xor_cipher(hex_string, key):
    hex_bytes = bytes.fromhex(hex_string)
    # XOR each byte with the key (ASCII value of the provided key)

    decrypted_bytes = bytes([byte ^ key for byte in hex_bytes])

    # Convert the result to a string
    decrypted_text = decrypted_bytes.decode('utf-8', errors='ignore')

    return decrypted_text



def score_decryption(text):
    score = 0
    for char in text:
        ascii_val = ord(char)
        #Penalize Characters that are strange for a meaningful English sentence such as /, \, #
        if ascii_val <= 30 or (ascii_val >= 33 and ascii_val <= 47) or ascii_val == 92 or ascii_val == 47:
            score -= 1  
        #Reward spaces, as they are very common in sentences
        if ascii_val == 32:
            score += 2
        #Reward lowercase and uppercase letters
        elif (ascii_val >= 65 and ascii_val <= 90) or (ascii_val >= 97 and ascii_val <= 122) or ascii_val == 39 or (ascii_val >= 48 and ascii_val <= 57):
            score += 1
        else:
            score -= 1

    return score


def xor_cipher_solver(text):
    topscore = 0
    plaintext = ''
    bestkey = ''
    for key in range (2**8):
        decrypted_message = single_byte_xor_cipher(text, key)
        score = score_decryption(decrypted_message)
        if score > topscore:
            plaintext = decrypted_message
            topscore = score
            bestkey = key       

    return (plaintext, chr(bestkey), topscore)

hex_encoded_message = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
result = xor_cipher_solver(hex_encoded_message)

print(f"Plaintext: {result[0]} Key: {result[1]}")

Plaintext: Cooking MC's like a pound of bacon Key: X


## Step 4 - Detect single-character XOR

> One of the 60-character strings in [this file](data/04.txt) has been encrypted by single-character XOR (each line is one string).
>
> Find it.
>
> ### Comment
> You should use your code in Step 3 to test each line. One line should output a meaningful message. Remeber that you don't know the key either but you can find it for each line (if any). 

#### Description
*A brief description of your approach. Don't just put the code. First explain what you did and WHY you did it!*

<p> (your description)<br>
I took the text file and put each line into a Python list stripping out the newlines. Then I could simply use the code from step 3 to find the best possible key for each string, and then return the string and key that had the highest score out of all the strings.
</p>

In [279]:
#Open Text File
with open("data/04.txt", 'r') as file:
    lines = file.readlines()
#Get list of hex strings
lines = [line.strip() for line in lines]
best = 0
solved = ()
#Keep track of highest scoring string and return it
for line in lines:
    temp = xor_cipher_solver(line)
    if temp[-1] > best:
        best = temp[-1]
        solved = temp
print(f"Plaintext: {solved[0]} Key: {solved[1]}")

Plaintext: Now that the party is jumping
 Key: 5


## Step 5 - Implement repeating-key XOR

> Here is the opening stanza of an important work of the English language:
> ```
> Burning 'em, if you ain't quick and nimble
> I go crazy when I hear a cymbal
> ```
> Encrypt it, under the key "ICE", using repeating-key XOR.
>
> In repeating-key XOR, you'll sequentially apply each byte of the key; the first byte of plaintext will be XOR'd against I, the next C, the next E, then I again for the 4th byte, and so on.
>
> It should come out to:
> ```
> 0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272
> a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f
> ```


In [255]:
def repeating_xor_encrypt(plaintext, key):
    # Ensure key is not empty
    if not key:
        raise ValueError("Key must not be empty")

    # Convert plaintext and key to bytes
    plaintext_bytes = plaintext.encode('utf-8')
    key_bytes = key.encode('utf-8')

    # XOR each byte of plaintext with the corresponding key byte
    encrypted_bytes = bytes([plaintext_byte ^ key_bytes[i % len(key_bytes)] for i, plaintext_byte in enumerate(plaintext_bytes)])

    # Convert the result to a hex string
    encrypted_hex = encrypted_bytes.hex()

    return encrypted_hex
repeating_xor_encrypt("Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal", "ICE")

'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

## Step 6 (Main Step) - Break repeating-key XOR

> There's a file [here](data/06.txt). It's been base64'd after being encrypted with repeating-key XOR.
>
> Decrypt it.
>
> Here's how:
>
> - Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40.
>
> - Write a function to compute the edit distance/Hamming distance between two strings. The Hamming distance is just the number of differing bits. The distance between:
```"this is a test"```
and
```"wokka wokka!!!"```
is 37. Make sure your code agrees before you proceed.
>
> - For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.
>
> - The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
>
> - Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
>
> - Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
>
> - Solve each block as if it was single-character XOR. You already have code to do this.
> For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.

#### Description
*A brief description of your approach. Don't just put the code. First explain what you did and WHY you did it!*

<p> (your description)<br>
I followed the steps for the most part. I first wanted to test all KEYSIZES from 2-40 as suggested, and then from there I took the first 4 KEYSIZED blocks and averaged their hamming distance. Then I found the 3 smallest KEYSIZEs and their corresponding hamming distance, and used them to bruteforce through the blocks. I transposed the blocks, and used that to solve each individual block with the single-character XOR method. From there I used my meaningful plaintext formula and returned the plaintext with the highest score as that was most likely the correct key.
</p>

In [427]:
# your code with comments

def single_byte_xor_cipher(hex_string, key):
    hex_bytes = bytes.fromhex(hex_string)
    # XOR each byte with the key (ASCII value of the provided key)
    decrypted_bytes = bytes([byte ^ key for byte in hex_bytes])

    # Convert the result to a string
    decrypted_text = decrypted_bytes.decode('utf-8', errors='ignore')

    return decrypted_text



def score_decryption(text):
    score = 0
    for char in text:
        ascii_val = ord(char)
        #Penalize Characters that are strange for a meaningful English sentence such as /, \, #
        if ascii_val <= 30 or (ascii_val >= 33 and ascii_val <= 47) or ascii_val == 92 or ascii_val == 47:
            score -= 1  
        #Reward spaces, as they are very common in sentences
        if ascii_val == 32:
            score += 2
        #Reward lowercase and uppercase letters
        elif (ascii_val >= 65 and ascii_val <= 90) or (ascii_val >= 97 and ascii_val <= 122) or ascii_val == 39 or (ascii_val >= 48 and ascii_val <= 57):
            score += 1
        else:
            score -= 1

    return score


def xor_cipher_solver(text):
    topscore = 0
    plaintext = ''
    bestkey = ''
    for key in range (2**8):
        decrypted_message = single_byte_xor_cipher(text, key)
        score = score_decryption(decrypted_message)
        if score > topscore:
            plaintext = decrypted_message
            topscore = score
            bestkey = key       
    return (plaintext, bestkey)


def hamming_distance(first, second):
    sum = 0
    count = 0
    while count < len(first):
        #When bits don't line up, we increment the Hamming DIstance by Definition
        if (first[count] != second[count]):
            sum += 1
        count += 1
    return sum

def repeating_xor_encrypt(plaintext, key):
    # Ensure key is not empty
    if not key:
        raise ValueError("Key must not be empty")

    # XOR each byte of plaintext with the corresponding key byte
    encrypted_bytes = bytes([plaintext_byte ^ key[i % len(key)] for i, plaintext_byte in enumerate(plaintext)])

    return encrypted_bytes

In [425]:
import itertools
import binascii
  
def repeating_xor_break(ciphertext):
    seen_distances = {}
    candidate_plaintexts = []
    for size in range(2, 40 + 1):
        # Grab the first four KEYSIZE blocks
        blocks = [ciphertext[counter:counter + size] for counter in range(0, len(ciphertext), size)][:4]
        combinations = itertools.combinations(blocks, 2)
        hamming = 0
        for (chunk1, chunk2) in combinations:
            hamming += hamming_distance(chunk1, chunk2)

        # Average out Hamming Distance (We have 6 distinct combinations that we compare)
        hamming = hamming / 6
        normal_distance = hamming / size
        seen_distances[size] = normal_distance

    # Find the three smallest hamming distances, and use the corresponding keys
    three_smallest = sorted(seen_distances, key=seen_distances.get)[:3]
    for key in three_smallest:
        # Split ciphertext into KEY_SIZE blocks and transpose them
        candidate_key = b''
        # Break apart blocks
        for s in range(key):
            split_block = b""
            # Find transpose of blocks
            for i in range(s, len(ciphertext), key):
                split_block += bytes([ciphertext[i]])
                #Solve each block to find the candidate key for each one
            candidate_key += bytes([xor_cipher_solver(binascii.hexlify(split_block).decode('utf-8'))[1]])

        candidate_plaintexts.append((repeating_xor_encrypt(ciphertext, candidate_key), candidate_key))
    #Return the decrypted plaintext that scores the highest on our meaningful sentence test
    return max(candidate_plaintexts, key=lambda x: score_decryption(x[0]))

# Load the ciphertext from the file
with open("data/06.txt", 'r') as file:
    ciphertext = b64decode(file.read())

# Call the function
result, key = repeating_xor_break(ciphertext)
#Ascii Encoding
ascii_string = bytes.fromhex(result).decode('utf-8')
key_string = bytes(key).decode('utf-8')
print("Key: ", key_string)
print("Plaintext: ",ascii_string)




Key:  Terminator X: Bring the noise
Plaintext:  I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you ca